In [ ]:
# Cellule 1 - Imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.genmod.families import Poisson
from statsmodels.genmod.families.family import Tweedie
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def load_data():
    print("=== Chargement des données ===")
    
    # 1. Chargement des données d'entraînement et de test
    print("\nChargement des données principales:")
    train_features = pd.read_csv('data/train_input_Z61KlZo.csv', low_memory=False)
    train_target = pd.read_csv('data/train_output_DzPxaPY.csv')
    test_df = pd.read_csv('data/test_input_5qJzHrr.csv', low_memory=False)
    
    # 2. Chargement du format de soumission
    print("\nChargement du format de soumission:")
    submission_format = pd.read_csv('data/submission_csv_file_random_example_3fbDtrr (1).csv')
    
    # Vérification des colonnes du format de soumission
    expected_cols = ['ID', 'FREQ', 'CM', 'ANNEE_ASSURANCE', 'CHARGE']
    if not all(col in submission_format.columns for col in expected_cols):
        raise ValueError(f"Format de soumission incorrect. Colonnes attendues: {expected_cols}")
    
    # 3. Gestion de la colonne ANNEE_ASSURANCE
    if 'ANNEE_ASSURANCE' in train_features.columns:
        train_features = train_features.drop('ANNEE_ASSURANCE', axis=1)
    
    # 4. Fusion des données train
    train_df = pd.merge(train_features, train_target, on='ID', how='inner')
    
    # 5. Vérifications et statistiques
    print("\n=== Dimensions des données ===")
    print(f"Train features initiales: {train_features.shape}")
    print(f"Train target: {train_target.shape}")
    print(f"Train après fusion: {train_df.shape}")
    print(f"Test: {test_df.shape}")
    print(f"Format soumission: {submission_format.shape}")
    
    # 6. Vérification de la cohérence des IDs
    test_ids = set(test_df['ID'])
    submission_ids = set(submission_format['ID'])
    
    if test_ids != submission_ids:
        print("\nATTENTION: Différence entre les IDs de test et de soumission!")
        print(f"IDs uniquement dans test: {len(test_ids - submission_ids)}")
        print(f"IDs uniquement dans soumission: {len(submission_ids - test_ids)}")
    
    # 7. Vérification des colonnes
    train_cols = set(train_df.columns) - {'FREQ', 'CM', 'CHARGE', 'ID'}
    test_cols = set(test_df.columns) - {'ID'}
    
    missing_cols = test_cols - train_cols
    extra_cols = train_cols - test_cols
    
    if missing_cols:
        print(f"\nColonnes manquantes dans train: {missing_cols}")
    if extra_cols:
        print(f"\nColonnes supplémentaires dans train: {extra_cols}")
    
    # 8. Vérification des types de données
    print("\nTypes de données dans train:")
    print(train_df.dtypes.value_counts())
    
    # 9. Création d'un template de soumission vide
    submission_template = submission_format.copy()
    submission_template[['FREQ', 'CM', 'CHARGE']] = 0
    
    return {
        'train_df': train_df,
        'test_df': test_df,
        'submission_format': submission_format,
        'submission_template': submission_template,
        'stats': {
            'train_shape': train_df.shape,
            'test_shape': test_df.shape,
            'submission_shape': submission_format.shape,
            'n_features': len(train_cols),
            'missing_cols': list(missing_cols),
            'extra_cols': list(extra_cols)
        }
    }

# Exécution du chargement
data_results = load_data()

# Récupération des DataFrames
train_df = data_results['train_df']
test_df = data_results['test_df']
submission_template = data_results['submission_template']

# Affichage des statistiques détaillées
print("\n=== Résumé du chargement ===")
stats = data_results['stats']
print(f"Nombre d'observations train: {stats['train_shape'][0]}")
print(f"Nombre d'observations test: {stats['test_shape'][0]}")
print(f"Nombre de features: {stats['n_features']}")

if stats['missing_cols']:
    print("\nAttention: colonnes manquantes dans train!")
    print(stats['missing_cols'])

if stats['extra_cols']:
    print("\nColonnes supplémentaires dans train:")
    print(stats['extra_cols'])

In [ ]:
# 1. Définition de tous les groupes de variables
WEATHER_VARS = {
    'temp_max': [col for col in train_df.columns if any(x in col.upper() for x in 
                 ['NBJTX', 'TX', 'TXMAX', 'TXAB']) and 'TMMAX' not in col.upper()],
    
    'temp_min': [col for col in train_df.columns if any(x in col.upper() for x in 
                 ['NBJTN', 'TN', 'TNMIN', 'TNAB']) and 'TMMIN' not in col.upper()],
    
    'temp_moy': [col for col in train_df.columns if any(x in col.upper() for x in 
                 ['TM', 'TMM', 'TMMAX', 'TMMIN'])],
    
    'temp_amplitude': [col for col in train_df.columns if 'TAMPLI' in col.upper()],
    
    'wind': [col for col in train_df.columns if any(x in col.upper() for x in 
             ['FF', 'FX'])],
    
    'rain': [col for col in train_df.columns if any(x in col.upper() for x in 
             ['RR', 'RRAB'])]
}

# Redéfinition des variables bâtiment et activité
BUILDING_VARS = {
    # Surfaces
    'surface': [f'SURFACE{i}' for i in range(1, 22)],
    
    # Nombre de bâtiments (attention au NBBAT12 manquant)
    'buildings': [f'NBBAT{i}' for i in range(1, 15) if i != 12],
    
    # Caractéristiques du bâtiment
    'characteristics': [f'CARACT{i}' for i in range(1, 6)],
    
    # Types de bâtiment
    'building_type': ['TYPBAT1', 'TYPBAT2', 'ADOSS'],
    
    # Hauteurs et données BDTOPO
    'height': [
        'HAUTEUR', 'HAUTEUR_MAX',
        'BDTOPO_BAT_MAX_HAUTEUR', 'BDTOPO_BAT_MAX_HAUTEUR_MAX'
    ]
}

ACTIVITY_VARS = {
    # Multi-équipement
    'equipment': [f'EQUIPEMENT{i}' for i in range(1, 8)],
    
    # Activité et vocation
    'activity': [
        'ACTIVIT2',     # Activité
        'VOCATION',     # Vocation de l'entité professionnelle
        'TYPERS'        # Type de personne
    ],
    
    # Catégorie
    'category': ['COEFASS'],  # Retrait des variables TAILLE qui sont dans INSURANCE_VARS
    
}

INSURANCE_VARS = {
    # Indicateurs de risque
    'risk_indicators': [f'RISK{i}' for i in range(1, 14)],
    
    # Données de capitaux
    'capital': [f'KAPITAL{i}' for i in range(1, 44)],
    
    # Dérogations tarifaires
    'derogation': [f'DEROG{i}' for i in range(1, 17)],
    
    # Franchises et indemnisations
    'contract_terms': {
        'franchise': ['FRCH1', 'FRCH2'],
        'indemnisation': ['INDEM1', 'INDEM2']
    },
    
    # Données d'activité et taille
    'business_profile': {
        'chiffre_affaires': ['CA1', 'CA2', 'CA3'],
        'taille_risque': [f'TAILLE{i}' for i in range(1, 5)]  # Gardé ici car lié au profil business
    },
    
    # Historique et sinistralité
    'history': [
        'NBSINCONJ',     # Sinistralité conjoncturelle
        'NBSINSTRT',     # Sinistralité longue période
        'ANCIENNETE',    # Ancienneté du contrat
        'DUREE_REQANEUF' # Durée
    ],
    
    # Données temporelles
    'time': [
        'AN_EXERC',        # Année d'exercice
        'ANNEE_ASSURANCE'  # Nombre d'années assurées
    ]
}
GEOGRAPHIC_VARS = {
    # Distances par type d'environnement
    'distances': {
        'urban': [  # Zones urbaines
            'DISTANCE_111',  # Tissu urbain continu
            'DISTANCE_112',  # Tissu urbain discontinu
            'DISTANCE_121',  # Zones industrielles/commerciales
            'DISTANCE_122',  # Réseaux routier/ferroviaire
            'DISTANCE_123',  # Zones portuaires
            'DISTANCE_124',  # Aéroports
            'DISTANCE_131',  # Extraction de matériaux
            'DISTANCE_132',  # Décharges
            'DISTANCE_133',  # Chantiers
            'DISTANCE_141',  # Espaces verts urbains
            'DISTANCE_142'   # Équipements sportifs et de loisirs
        ],
        'natural': [  # Zones naturelles
            'DISTANCE_311',  # Forêts de feuillus
            'DISTANCE_312',  # Forêts de conifères
            'DISTANCE_313',  # Forêts mélangées
            'DISTANCE_321',  # Pelouses et pâturages naturels
            'DISTANCE_322',  # Landes et broussailles
            'DISTANCE_323',  # Végétation sclérophylle
            'DISTANCE_324',  # Forêt et végétation arbustive en mutation
            'DISTANCE_331',  # Plages, dunes et sable
            'DISTANCE_332',  # Roches nues
            'DISTANCE_333',  # Végétation clairsemée
            'DISTANCE_334',  # Zones incendiées
            'DISTANCE_335'   # Glaciers et neiges éternelles
        ],
        'water': [  # Zones aquatiques
            'DISTANCE_411',  # Marais intérieurs
            'DISTANCE_412',  # Tourbières
            'DISTANCE_421',  # Marais maritimes
            'DISTANCE_422',  # Marais salants
            'DISTANCE_423',  # Zones intertidales
            'DISTANCE_511',  # Cours d'eau
            'DISTANCE_512',  # Plans d'eau
            'DISTANCE_521',  # Lagunes littorales
            'DISTANCE_522',  # Estuaires
            'DISTANCE_523'   # Mers et océans
        ],
        'other': [  # Autres distances
            'DISTANCE_211',  # Terres arables hors périmètres d'irrigation
            'DISTANCE_212',  # Périmètres irrigués en permanence
            'DISTANCE_213',  # Rizières
            'DISTANCE_221',  # Vignobles
            'DISTANCE_222',  # Vergers et petits fruits
            'DISTANCE_223',  # Oliveraies
            'DISTANCE_231',  # Prairies et autres surfaces toujours en herbe
            'DISTANCE_242',  # Systèmes culturaux et parcellaires complexes
            'DISTANCE_243',  # Surfaces agricoles avec végétation naturelle
            'DISTANCE_244'   # Territoires agroforestiers
        ],
        'general': [  # Distances générales
            'DISTANCE_1',    # Distance générale 1
            'DISTANCE_2'     # Distance générale 2
        ]
    },
    
    # Proportions par type d'environnement
    'proportions': {
        'urban': [  # Zones urbaines
            'PROPORTION_11',  # Zones urbanisées
            'PROPORTION_12',  # Zones industrielles et commerciales
            'PROPORTION_13',  # Mines, décharges et chantiers
            'PROPORTION_14'   # Espaces verts artificialisés
        ],
        'agricultural': [  # Zones agricoles
            'PROPORTION_21',  # Terres arables
            'PROPORTION_22',  # Cultures permanentes
            'PROPORTION_23',  # Prairies
            'PROPORTION_24'   # Zones agricoles hétérogènes
        ],
        'natural': [  # Zones naturelles
            'PROPORTION_31',  # Forêts
            'PROPORTION_32',  # Milieux à végétation arbustive et/ou herbacée
            'PROPORTION_33'   # Espaces ouverts, sans ou avec peu de végétation
        ],
        'water': [  # Zones aquatiques
            'PROPORTION_41',  # Zones humides intérieures
            'PROPORTION_42',  # Zones humides maritimes
            'PROPORTION_51',  # Eaux continentales
            'PROPORTION_52'   # Eaux maritimes
        ]
    },
    
    # Données d'altitude et zones
    'topography': {
        'altitude': [f'ALTITUDE_{i}' for i in range(1, 6)],  # Différents niveaux d'altitude
        'zone_info': [
            'ZONE',          # Zone géographique
            'ZONE_VENT',     # Zone de vent
            'ESPINSEE'       # Espace INSEE
        ]
    },
    
    # Données d'urgence
    'emergency': ['NB_CASERNES']  # Nombre de casernes de pompiers à proximité
}
DEMOGRAPHIC_VARS = {
    # Variables ménages
    'household': {
        'general': ['MEN'],  # Nombre total de ménages
        'composition': [
            'MEN_1IND',      # Ménages d'un seul individu
            'MEN_5IND',      # Ménages de 5 individus ou plus
            'MEN_FMP'        # Ménages monoparentaux
        ],
        'housing_type': [
            'MEN_COLL',      # Ménages en logements collectifs
            'MEN_MAIS',      # Ménages en maison
            'MEN_PROP'       # Ménages propriétaires
        ],
        'economic': [
            'MEN_PAUV',      # Ménages pauvres
            'MEN_SURF'       # Surface des logements du carreau
        ]
    },
    
    # Variables logement
    'housing': {
        'construction_period': [
            'LOG_AVA1',      # Avant année A1
            'LOG_A1_A2',     # Entre A1 et A2
            'LOG_A2_A3',     # Entre A2 et A3
            'LOG_APA3'       # Après A3
        ],
        'special_categories': [
            'LOG_INC',       # Date de construction inconnue
            'LOG_SOC'        # Logements sociaux
        ]
    },
    
    # Variables individus
    'individual': {
        'total': ['IND'],    # Nombre total d'individus
        'age_groups': [
            'IND_0_Y1',      # 0 à Y1 ans
            'IND_Y1_Y2',     # Y1 à Y2 ans
            'IND_Y2_Y3',     # Y2 à Y3 ans
            'IND_Y3_Y4',     # Y3 à Y4 ans
            'IND_Y4_Y5',     # Y4 à Y5 ans
            'IND_Y5_Y6',     # Y5 à Y6 ans
            'IND_Y6_Y7',     # Y6 à Y7 ans
            'IND_Y7_Y8',     # Y7 à Y8 ans
            'IND_Y8_Y9',     # Y8 à Y9 ans
            'IND_Y9'         # Y9 ans ou plus
        ],
        'other': [
            'IND_INC',       # Âge inconnu
            'IND_SNV'        # Niveaux de vie winsorisés
        ]
    }
}
TARGET_VARS = {
    'primary': {
        'frequency': ['FREQ'],      # Fréquence
        'cost': ['CM'],            # Coût moyen
        'total': ['CHARGE'],       # Charge totale
        'period': ['ANNEE_ASSURANCE']  # Période d'assurance
    },
    'definitions': {
        'FREQ': 'NOMBRE DE SINISTRES / ANNEE ASSURANCE',
        'CM': 'CHARGE SINISTRES / NOMBRE DE SINISTRES',
        'CHARGE': 'FREQ * CM * ANNEE_ASSURANCE',
        'ANNEE_ASSURANCE': 'Durée de la période d\'assurance en années'
    }
}
ID_VARS = {
    'identifiers': ['ID'],
    'properties': {
        'unique': True,          # L'ID doit être unique
        'not_null': True,        # Pas de valeurs nulles autorisées
        'type': 'str'           # Type attendu
    }
}



In [ ]:
def verify_weather_vars():
    print("=== Vérification des variables météo ===\n")
    
    # 1. Vérifier les doublons globaux
    all_weather_vars = []
    for category, cols in WEATHER_VARS.items():
        all_weather_vars.extend(cols)
    
    duplicates = [var for var in set(all_weather_vars) 
                 if all_weather_vars.count(var) > 1]
    
    if duplicates:
        print("ATTENTION! Variables en double:")
        for var in duplicates:
            categories = [cat for cat, cols in WEATHER_VARS.items() if var in cols]
            print(f"- {var} présent dans: {categories}")
    
    # 2. Vérifier chaque catégorie
    for category, cols in WEATHER_VARS.items():
        print(f"\n{category.upper()}:")
        print(f"Nombre de variables: {len(cols)}")
        
        # Vérifier les préfixes
        prefixes = set([col.split('_')[0] for col in cols])
        print("Préfixes trouvés:", prefixes)
        
        # Exemples de variables
        print("Exemples de variables:", sorted(cols)[:3])
        
        # Vérifier l'existence dans le DataFrame
        missing_in_df = [col for col in cols if col not in train_df.columns]
        if missing_in_df:
            print(f"ATTENTION! Variables non trouvées dans le DataFrame:")
            print(missing_in_df)
        
        # Vérifier les valeurs manquantes
        existing_cols = [col for col in cols if col in train_df.columns]
        if existing_cols:
            missing_pct = (train_df[existing_cols].isnull().mean() * 100).mean()
            print(f"Pourcentage moyen de valeurs manquantes: {missing_pct:.2f}%")
    
    # 3. Vérifier si des variables météo n'ont pas été classées
    weather_patterns = {
        'temp': ['TX', 'TN', 'TM', 'TEMP'],
        'wind': ['FF', 'FX'],
        'rain': ['RR', 'RRAB'],
        'amplitude': ['TAMPLI']
    }
    
    unclassified = []
    for pattern_type, patterns in weather_patterns.items():
        for col in train_df.columns:
            if any(pattern in col.upper() for pattern in patterns):
                if col not in all_weather_vars:
                    unclassified.append((col, pattern_type))
    
    if unclassified:
        print("\nVariables météo potentiellement non classées:")
        for col, type_ in unclassified:
            print(f"- {col} (type: {type_})")
    
    # 4. Résumé
    print("\n=== Résumé des variables météo ===")
    print(f"Total variables classées: {len(set(all_weather_vars))}")
    print(f"Variables en double: {len(duplicates)}")
    print(f"Variables non classées: {len(unclassified)}")
    
    return {
        'classified_vars': set(all_weather_vars),
        'duplicates': duplicates,
        'unclassified': unclassified,
        'missing_in_df': {cat: [col for col in cols if col not in train_df.columns] 
                         for cat, cols in WEATHER_VARS.items()}
    }

# Exécuter la vérification
weather_results = verify_weather_vars()

In [ ]:
def verify_building_and_activity_vars():
    print("=== Vérification des variables bâtiment et activité ===\n")
    
    # Collecter toutes les variables pour vérification des doublons
    all_building_vars = []
    all_activity_vars = []
    
    # Vérifier les deux groupes
    for group_name, group_vars, all_vars_list in [
        ("BUILDING", BUILDING_VARS, all_building_vars), 
        ("ACTIVITY", ACTIVITY_VARS, all_activity_vars)
    ]:
        print(f"\n{group_name}:")
        
        for category, vars_list in group_vars.items():
            print(f"\n  {category}:")
            print(f"  Nombre de variables définies: {len(vars_list)}")
            print("  Variables:", vars_list)
            
            # Vérifier l'existence
            existing_vars = [var for var in vars_list if var in train_df.columns]
            missing_vars = [var for var in vars_list if var not in train_df.columns]
            
            print(f"  Variables trouvées: {len(existing_vars)}/{len(vars_list)}")
            
            if missing_vars:
                print(f"  ATTENTION! Variables manquantes: {missing_vars}")
            
            # Vérifier les valeurs manquantes pour les variables existantes
            if existing_vars:
                missing_pct = (train_df[existing_vars].isnull().mean() * 100).mean()
                print(f"  Pourcentage moyen de valeurs manquantes: {missing_pct:.2f}%")
            
            all_vars_list.extend(vars_list)
    
    # Vérifier les doublons entre les deux groupes
    building_vars = set(all_building_vars)
    activity_vars = set(all_activity_vars)
    duplicates = building_vars.intersection(activity_vars)
    
    if duplicates:
        print("\nATTENTION! Variables en double entre BUILDING et ACTIVITY:")
        print(duplicates)
    
    # 3. Vérifier les variables non classées par type
    patterns = {
        'surface': 'SURFACE',
        'batiment': 'NBBAT',
        'caracteristique': 'CARACT',
        'type_batiment': 'TYPBAT',
        'hauteur': 'HAUTEUR',
        'bdtopo': 'BDTOPO',
        'equipement': 'EQUIPEMENT',
        'activite': 'ACTIVIT',
        'vocation': 'VOCATION',
        'type_personne': 'TYPERS'
    }
    
    unclassified = {}
    all_defined_vars = building_vars.union(activity_vars)
    
    for pattern_name, pattern in patterns.items():
        pattern_vars = [col for col in train_df.columns if pattern in col 
                       and col not in all_defined_vars]
        if pattern_vars:
            unclassified[pattern_name] = pattern_vars
    
    if unclassified:
        print("\nVariables non classées par type:")
        for pattern_name, vars_list in unclassified.items():
            print(f"\n{pattern_name.upper()}:")
            print(f"Nombre: {len(vars_list)}")
            print("Variables:", vars_list)
    
    # Résumé final
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    
    print("\n=== Résumé ===")
    print(f"Variables bâtiment: {len(building_vars)}")
    print(f"Variables activité: {len(activity_vars)}")
    print(f"Variables en double: {len(duplicates)}")
    print(f"Variables non classées: {total_unclassified}")
    
    return {
        'building_vars': building_vars,
        'activity_vars': activity_vars,
        'duplicates': duplicates,
        'unclassified': unclassified,
        'stats': {
            'building': {cat: len(vars_) for cat, vars_ in BUILDING_VARS.items()},
            'activity': {cat: len(vars_) for cat, vars_ in ACTIVITY_VARS.items()},
            'total_unclassified': total_unclassified
        }
    }

# Exécuter la vérification
building_activity_results = verify_building_and_activity_vars()

In [ ]:
import re  # Pour les expressions régulières
def verify_insurance_vars():
    """
    Vérifie la cohérence et la présence des variables d'assurance.
    """
    print("=== Vérification des variables d'assurance ===\n")
    
    # 1. Collecter toutes les variables définies
    all_defined_vars = []
    
    # 2. Vérifier chaque catégorie principale
    for category, content in INSURANCE_VARS.items():
        # 2.1 Traitement des catégories avec sous-catégories
        if isinstance(content, dict):
            print(f"\n{category.upper()}:")
            for subcat, vars_list in content.items():
                print(f"\n  {subcat}:")
                verify_variable_group(vars_list, all_defined_vars)
        
        # 2.2 Traitement des catégories simples
        else:
            print(f"\n{category.upper()}:")
            verify_variable_group(content, all_defined_vars)
    
    # 3. Vérifier les doublons
    duplicates = find_duplicates(all_defined_vars)
    
    # 4. Vérifier les variables non classées
    unclassified = find_unclassified_vars(all_defined_vars)
    
    # 5. Générer le résumé
    print_summary(all_defined_vars, duplicates, unclassified)
    
    return generate_results(all_defined_vars, duplicates, unclassified)

def verify_variable_group(vars_list, all_vars):
    """
    Vérifie un groupe de variables et met à jour la liste globale.
    """
    print(f"  Nombre de variables définies: {len(vars_list)}")
    print("  Variables:", vars_list)
    
    # Vérifier l'existence
    existing_vars = [var for var in vars_list if var in train_df.columns]
    missing_vars = [var for var in vars_list if var not in train_df.columns]
    
    print(f"  Variables trouvées: {len(existing_vars)}/{len(vars_list)}")
    
    if missing_vars:
        print(f"  ATTENTION! Variables manquantes: {missing_vars}")
    
    # Vérifier les valeurs manquantes
    if existing_vars:
        missing_pct = (train_df[existing_vars].isnull().mean() * 100).mean()
        print(f"  Pourcentage moyen de valeurs manquantes: {missing_pct:.2f}%")
    
    all_vars.extend(vars_list)

def find_duplicates(all_vars):
    """
    Identifie les variables en double.
    """
    return [var for var in set(all_vars) if all_vars.count(var) > 1]

def find_unclassified_vars(all_vars):
    """
    Identifie les variables d'assurance non classées.
    """
    insurance_patterns = {
        'risk': '^RISK[0-9]+$',
        'capital': '^KAPITAL[0-9]+$',
        'derogation': '^DEROG[0-9]+$',
        'franchise': '^FRCH[0-9]+$',
        'indemnisation': '^INDEM[0-9]+$',
        'chiffre_affaires': '^CA[0-9]+$',
        'sinistre': '^NBSIN',
        'anciennete': '^ANCIEN',
        'exercice': '^(AN_)?EXERC'
    }
    
    # Exclure les patterns d'autres groupes
    other_patterns = ['VOCATION', 'CARACT', 'NB_CASERNES', 'EQUIPEMENT', 'SURFACE']
    
    unclassified = {}
    for pattern_name, pattern in insurance_patterns.items():
        pattern_vars = [col for col in train_df.columns 
                       if re.match(pattern, col) and col not in all_vars
                       and not any(other in col for other in other_patterns)]
        if pattern_vars:
            unclassified[pattern_name] = pattern_vars
    
    return {k: v for k, v in unclassified.items() if v}

def print_summary(all_vars, duplicates, unclassified):
    """
    Affiche le résumé de la vérification.
    """
    if unclassified:
        print("\nVariables non classées par type:")
        for pattern_name, vars_list in unclassified.items():
            print(f"\n{pattern_name.upper()}:")
            print(f"Nombre: {len(vars_list)}")
            print("Variables:", vars_list)
    
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    
    print("\n=== Résumé ===")
    print(f"Total variables définies: {len(all_vars)}")
    print(f"Variables uniques: {len(set(all_vars))}")
    print(f"Doublons: {len(duplicates)}")
    if duplicates:
        print("Variables en double:", duplicates)
    print(f"Variables non classées: {total_unclassified}")

def generate_results(all_vars, duplicates, unclassified):
    """
    Génère le dictionnaire de résultats.
    """
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    
    return {
        'all_vars': set(all_vars),
        'duplicates': duplicates,
        'unclassified': unclassified,
        'missing_vars': {cat: [var for var in (content if isinstance(content, list) 
                                              else sum(content.values(), [])) 
                              if var not in train_df.columns] 
                        for cat, content in INSURANCE_VARS.items()},
        'stats': {
            'total_defined': len(all_vars),
            'total_unique': len(set(all_vars)),
            'total_duplicates': len(duplicates),
            'total_unclassified': total_unclassified
        }
    }

# Exécuter la vérification
insurance_results = verify_insurance_vars()

In [ ]:
def verify_geographic_vars():
    print("=== Vérification des variables géographiques ===\n")
    
    # Aplatir la structure imbriquée
    all_vars = []
    category_vars = {}  # Pour suivre les variables par catégorie
    
    # 1. Collecter toutes les variables
    for main_category, content in GEOGRAPHIC_VARS.items():
        category_vars[main_category] = []
        if isinstance(content, dict):
            for subcategory, vars_list in content.items():
                all_vars.extend(vars_list)
                category_vars[main_category].extend(vars_list)
        else:
            all_vars.extend(content)
            category_vars[main_category].extend(content)
    
    # 2. Vérifier chaque catégorie
    for main_category, content in GEOGRAPHIC_VARS.items():
        print(f"\n{main_category.upper()}:")
        
        if isinstance(content, dict):
            for subcategory, vars_list in content.items():
                print(f"\n  {subcategory}:")
                print(f"  Nombre de variables définies: {len(vars_list)}")
                if len(vars_list) > 3:
                    print(f"  Exemples de variables: {vars_list[:3]} ...")
                else:
                    print(f"  Variables: {vars_list}")
                
                # Vérifier l'existence
                existing_vars = [var for var in vars_list if var in train_df.columns]
                missing_vars = [var for var in vars_list if var not in train_df.columns]
                
                print(f"  Variables trouvées: {len(existing_vars)}/{len(vars_list)}")
                
                if missing_vars:
                    print(f"  ATTENTION! Variables manquantes ({len(missing_vars)}):")
                    for var in missing_vars[:5]:  # Montrer les 5 premiers exemples
                        print(f"    - {var}")
                    if len(missing_vars) > 5:
                        print(f"    ... et {len(missing_vars)-5} autres")
                
                # Vérifier les valeurs manquantes
                if existing_vars:
                    missing_pct = (train_df[existing_vars].isnull().mean() * 100).mean()
                    print(f"  Pourcentage moyen de valeurs manquantes: {missing_pct:.2f}%")
        else:
            print(f"Nombre de variables: {len(content)}")
            print("Variables:", content)
    
    # 3. Vérifier les variables non classées
    patterns = {
        'distance': 'DISTANCE_',
        'proportion': 'PROPORTION_',
        'altitude': 'ALTITUDE_',
        'zone': 'ZONE'
    }
    
    unclassified = {}
    for pattern_name, pattern in patterns.items():
        pattern_vars = [col for col in train_df.columns if pattern in col 
                       and col not in all_vars]
        if pattern_vars:
            unclassified[pattern_name] = pattern_vars
    
    if unclassified:
        print("\nVariables non classées par type:")
        for pattern_name, vars_list in unclassified.items():
            print(f"\n{pattern_name.upper()}:")
            print(f"Nombre: {len(vars_list)}")
            print("Exemples:", vars_list[:5], "..." if len(vars_list) > 5 else "")
    
    # 4. Résumé final
    print("\n=== Résumé de la vérification géographique ===")
    print(f"Total variables définies: {len(all_vars)}")
    
    existing_total = len([var for var in all_vars if var in train_df.columns])
    print(f"Variables existantes: {existing_total}")
    print(f"Variables manquantes: {len(all_vars) - existing_total}")
    
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    print(f"Variables non classées: {total_unclassified}")
    
    # 5. Statistiques par catégorie
    print("\nStatistiques par catégorie principale:")
    for category, vars_list in category_vars.items():
        existing = len([var for var in vars_list if var in train_df.columns])
        print(f"{category}: {existing}/{len(vars_list)} variables trouvées")
    
    return {
        'all_vars': set(all_vars),
        'by_category': category_vars,
        'unclassified': unclassified,
        'stats': {
            'total_defined': len(all_vars),
            'total_existing': existing_total,
            'total_missing': len(all_vars) - existing_total,
            'total_unclassified': total_unclassified
        }
    }

# Exécuter la vérification
geo_results = verify_geographic_vars()

In [ ]:
def verify_demographic_vars():
    print("=== Vérification des variables démographiques ===\n")
    
    # Aplatir la structure imbriquée
    all_vars = []
    for main_category, subcategories in DEMOGRAPHIC_VARS.items():
        for subcat, vars_list in subcategories.items():
            all_vars.extend(vars_list)
    
    # Vérifier chaque catégorie et sous-catégorie
    for main_category, subcategories in DEMOGRAPHIC_VARS.items():
        print(f"\n{main_category.upper()}:")
        
        for subcat, vars_list in subcategories.items():
            print(f"\n  {subcat}:")
            print(f"  Nombre de variables définies: {len(vars_list)}")
            print("  Variables définies:", vars_list)
            
            # Vérifier l'existence
            existing_vars = [var for var in vars_list if var in train_df.columns]
            missing_vars = [var for var in vars_list if var not in train_df.columns]
            
            print(f"  Variables trouvées: {len(existing_vars)}/{len(vars_list)}")
            
            if missing_vars:
                print(f"  ATTENTION! Variables manquantes: {missing_vars}")
            
            # Vérifier les valeurs manquantes seulement pour les variables existantes
            if existing_vars:
                missing_pct = (train_df[existing_vars].isnull().mean() * 100).mean()
                print(f"  Pourcentage moyen de valeurs manquantes: {missing_pct:.2f}%")
    
    # Patterns pour identifier les variables démographiques
    demo_patterns = {
        'menage': '^MEN_',
        'logement': '^LOG_',
        'individu': '^IND_'
    }
    
    # Exclure les patterns d'autres groupes
    other_patterns = ['INDEM', 'EQUIPEMENT', 'ACTIVIT', 'SURFACE', 'CARACT']
    
    # Vérifier les variables non classées
    unclassified = {}
    for pattern_name, pattern in demo_patterns.items():
        pattern_vars = [col for col in train_df.columns 
                       if re.match(pattern, col) 
                       and col not in all_vars
                       and not any(other in col for other in other_patterns)]
        if pattern_vars:
            unclassified[pattern_name] = pattern_vars
    
    # Afficher les variables non classées par type
    if unclassified:
        print("\nVariables non classées par type:")
        for pattern_name, vars_list in unclassified.items():
            if vars_list:  # N'afficher que si la liste n'est pas vide
                print(f"\n{pattern_name.upper()}:")
                print(f"Nombre: {len(vars_list)}")
                print("Variables:", vars_list)
    
    # Résumé final
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    
    print("\n=== Résumé de la vérification démographique ===")
    existing_vars = [var for var in all_vars if var in train_df.columns]
    missing_vars = [var for var in all_vars if var not in train_df.columns]
    print(f"Variables définies: {len(all_vars)}")
    print(f"Variables existantes: {len(existing_vars)}")
    print(f"Variables manquantes: {len(missing_vars)}")
    print(f"Variables non classées: {total_unclassified}")
    
    return {
        'all_defined_vars': all_vars,
        'existing_vars': existing_vars,
        'missing_vars': missing_vars,
        'unclassified': unclassified,
        'stats': {
            'total_defined': len(all_vars),
            'total_existing': len(existing_vars),
            'total_missing': len(missing_vars),
            'total_unclassified': total_unclassified
        }
    }

# Exécuter la vérification
demo_results = verify_demographic_vars()

In [ ]:
def verify_target_vars():
    print("=== Vérification des variables cibles ===\n")
    
    # Aplatir la structure pour les variables
    all_vars = []
    for category in TARGET_VARS['primary'].values():
        all_vars.extend(category)
    
    # Vérifier la présence des variables
    for category, vars_list in TARGET_VARS['primary'].items():
        print(f"\n{category.upper()}:")
        print(f"Variables: {vars_list}")
        
        # Vérifier l'existence
        missing = [var for var in vars_list if var not in train_df.columns]
        if missing:
            print(f"ATTENTION! Variables manquantes: {missing}")
        
        # Vérifier les valeurs manquantes
        if vars_list:
            missing_pct = (train_df[vars_list].isnull().mean() * 100).mean()
            print(f"Pourcentage moyen de valeurs manquantes: {missing_pct:.2f}%")
        
        # Afficher la définition
        for var in vars_list:
            if var in TARGET_VARS['definitions']:
                print(f"Définition de {var}: {TARGET_VARS['definitions'][var]}")
    
    # Vérifier les variables non classées
    target_patterns = {
        'frequence': '^FREQ',
        'cout_moyen': '^CM',
        'charge': '^CHARGE',
        'annee': '^ANNEE_ASSURANCE'
    }
    
    # Exclure les patterns d'autres groupes
    other_patterns = ['FRCH', 'CARACT', 'SURFACE']
    
    unclassified = {}
    for pattern_name, pattern in target_patterns.items():
        pattern_vars = [col for col in train_df.columns 
                       if re.match(pattern, col) 
                       and col not in all_vars
                       and not any(other in col for other in other_patterns)]
        if pattern_vars:
            unclassified[pattern_name] = pattern_vars
    
    if unclassified:
        print("\nVariables non classées par type:")
        for pattern_name, vars_list in unclassified.items():
            if vars_list:
                print(f"\n{pattern_name.upper()}:")
                print(f"Nombre: {len(vars_list)}")
                print("Variables:", vars_list)
    
    # Vérifier la cohérence des calculs
    if all(var in train_df.columns for var in ['FREQ', 'CM', 'CHARGE', 'ANNEE_ASSURANCE']):
        # Calculer CHARGE théorique
        charge_calc = train_df['FREQ'] * train_df['CM'] * train_df['ANNEE_ASSURANCE']
        
        # Comparer avec CHARGE réelle
        diff_pct = abs(charge_calc - train_df['CHARGE']).mean() / train_df['CHARGE'].mean() * 100
        print(f"\nDifférence moyenne entre CHARGE calculée et réelle: {diff_pct:.2f}%")
    
    # Statistiques descriptives
    print("\nStatistiques descriptives:")
    for var in all_vars:
        if var in train_df.columns:
            stats = train_df[var].describe()
            print(f"\n{var}:")
            print(f"  Min: {stats['min']:.2f}")
            print(f"  Max: {stats['max']:.2f}")
            print(f"  Moyenne: {stats['mean']:.2f}")
            print(f"  Écart-type: {stats['std']:.2f}")
    
    # Résumé final
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    print("\n=== Résumé ===")
    print(f"Variables définies: {len(all_vars)}")
    print(f"Variables existantes: {len([var for var in all_vars if var in train_df.columns])}")
    print(f"Variables manquantes: {len([var for var in all_vars if var not in train_df.columns])}")
    print(f"Variables non classées: {total_unclassified}")
    
    return {
        'variables': all_vars,
        'definitions': TARGET_VARS['definitions'],
        'unclassified': unclassified,
        'missing_stats': {var: train_df[var].isnull().mean() * 100 
                         for var in all_vars if var in train_df.columns},
        'stats': {
            'total_defined': len(all_vars),
            'total_existing': len([var for var in all_vars if var in train_df.columns]),
            'total_missing': len([var for var in all_vars if var not in train_df.columns]),
            'total_unclassified': total_unclassified
        }
    }

# Exécuter la vérification
target_results = verify_target_vars()

In [ ]:
def verify_id_vars():
    """
    Vérifie la cohérence et la présence des variables d'identification.
    """
    print("=== Vérification des variables d'identification ===\n")
    
    id_vars = ID_VARS['identifiers']
    properties = ID_VARS['properties']
    
    # 1. Vérifier la présence des variables
    missing = [var for var in id_vars if var not in train_df.columns]
    if missing:
        print(f"ATTENTION! Variables manquantes: {missing}")
        return
    
    # 2. Vérifier chaque identifiant
    for id_var in id_vars:
        verify_single_id(id_var, properties)
    
    # 3. Vérifier les variables non classées
    unclassified = find_unclassified_vars(id_vars)
    
    # 4. Afficher les statistiques générales
    print_id_statistics(id_vars)
    
    # 5. Générer et afficher le résumé final
    results = generate_results(id_vars, unclassified)
    print_final_summary(results)
    
    return results

def verify_single_id(id_var, properties):
    """Vérifie un identifiant spécifique."""
    print(f"\nVérification de {id_var}:")
    
    # Vérifier l'unicité
    if properties['unique']:
        check_uniqueness(id_var)
    
    # Vérifier les valeurs nulles
    if properties['not_null']:
        check_null_values(id_var)
    
    # Vérifier le type de données
    if properties['type'] == 'str':
        check_data_type(id_var)
    
    # Vérifier la cohérence entre les ensembles
    check_dataset_consistency(id_var)

def check_uniqueness(id_var):
    """Vérifie l'unicité des identifiants."""
    n_unique = train_df[id_var].nunique()
    n_total = len(train_df)
    if n_unique != n_total:
        print(f"ATTENTION! {id_var} n'est pas unique:")
        print(f"  {n_unique} valeurs uniques pour {n_total} lignes")
        duplicates = train_df[train_df[id_var].duplicated(keep=False)]
        if not duplicates.empty:
            print("  Exemples de doublons:")
            print(duplicates[id_var].head())

def check_null_values(id_var):
    """Vérifie la présence de valeurs nulles."""
    n_null = train_df[id_var].isnull().sum()
    if n_null > 0:
        print(f"ATTENTION! {n_null} valeurs nulles trouvées")

def check_data_type(id_var):
    """Vérifie le type de données."""
    if not pd.api.types.is_string_dtype(train_df[id_var]):
        print(f"ATTENTION! {id_var} n'est pas de type string")
        print(f"Type actuel: {train_df[id_var].dtype}")

def check_dataset_consistency(id_var):
    """Vérifie la cohérence entre train, test et soumission."""
    train_ids = set(train_df[id_var])
    test_ids = set(test_df[id_var])
    submission_ids = set(data_results['submission_format'][id_var])
    
    # Vérifier l'intersection train/test
    common_train_test = train_ids.intersection(test_ids)
    if common_train_test:
        print(f"\nATTENTION! IDs communs entre train et test: {len(common_train_test)}")
        print("Exemples:", list(common_train_test)[:5])
    
    # Vérifier la cohérence test/soumission
    check_submission_consistency(test_ids, submission_ids)
    
    return train_ids, test_ids, submission_ids, common_train_test

def check_submission_consistency(test_ids, submission_ids):
    """Vérifie la cohérence avec le fichier de soumission."""
    missing_in_submission = test_ids - submission_ids
    extra_in_submission = submission_ids - test_ids
    if missing_in_submission or extra_in_submission:
        print("\nIncohérences entre test et soumission:")
        if missing_in_submission:
            print(f"IDs manquants dans soumission: {len(missing_in_submission)}")
        if extra_in_submission:
            print(f"IDs supplémentaires dans soumission: {len(extra_in_submission)}")
    return missing_in_submission, extra_in_submission

def find_unclassified_vars(id_vars):
    """Identifie les variables d'identification non classées."""
    id_patterns = {
        'identifiant': '^ID',
        'reference': '^REF',
        'numero': '^NUM'
    }
    
    # Exclure les patterns d'autres groupes
    other_patterns = ['INDEM', 'IND_', 'INDICE']
    
    unclassified = {}
    for pattern_name, pattern in id_patterns.items():
        pattern_vars = [col for col in train_df.columns 
                       if re.match(pattern, col) 
                       and col not in id_vars
                       and not any(other in col for other in other_patterns)]
        if pattern_vars:
            unclassified[pattern_name] = pattern_vars
    
    if unclassified:
        print("\nVariables non classées par type:")
        for pattern_name, vars_list in unclassified.items():
            if vars_list:
                print(f"\n{pattern_name.upper()}:")
                print(f"Nombre: {len(vars_list)}")
                print("Variables:", vars_list)
    
    return unclassified

def print_id_statistics(id_vars):
    """Affiche les statistiques des identifiants."""
    print("\n=== Statistiques des IDs ===")
    for id_var in id_vars:
        print(f"\n{id_var}:")
        print(f"Train: {train_df[id_var].nunique()} valeurs uniques")
        print(f"Test: {test_df[id_var].nunique()} valeurs uniques")
        print(f"Soumission: {data_results['submission_format'][id_var].nunique()} valeurs uniques")
        
        if pd.api.types.is_string_dtype(train_df[id_var]):
            train_lengths = train_df[id_var].str.len()
            print(f"Longueur min: {train_lengths.min()}")
            print(f"Longueur max: {train_lengths.max()}")
            if train_lengths.min() != train_lengths.max():
                print("ATTENTION! Longueurs variables")

def generate_results(id_vars, unclassified):
    """Génère le dictionnaire des résultats."""
    id_var = id_vars[0]  # Premier identifiant
    train_ids = set(train_df[id_var])
    test_ids = set(test_df[id_var])
    submission_ids = set(data_results['submission_format'][id_var])
    
    total_unclassified = sum(len(vars_) for vars_ in unclassified.values())
    
    return {
        'train_ids': train_ids,
        'test_ids': test_ids,
        'submission_ids': submission_ids,
        'unclassified': unclassified,
        'stats': {
            'train_unique': len(train_ids),
            'test_unique': len(test_ids),
            'submission_unique': len(submission_ids),
            'total_unclassified': total_unclassified
        }
    }

def print_final_summary(results):
    """Affiche le résumé final."""
    print("\n=== Résumé final ===")
    stats = results['stats']
    print(f"Train: {stats['train_unique']} IDs uniques")
    print(f"Test: {stats['test_unique']} IDs uniques")
    print(f"Soumission: {stats['submission_unique']} IDs uniques")
    print(f"Variables non classées: {stats['total_unclassified']}")

# Exécuter la vérification
id_results = verify_id_vars()

In [ ]:
def verify_variable_classification():
    print("=== Vérification globale de la classification des variables ===")
    
    # 1. Collecter toutes les variables de chaque groupe
    group_vars = {
        'Weather': sum(WEATHER_VARS.values(), []),
        'Building': sum(BUILDING_VARS.values(), []),
        'Insurance': sum([v if isinstance(v, list) else sum(v.values(), []) 
                         for v in INSURANCE_VARS.values()], []),
        'Geographic': sum([v if isinstance(v, list) else sum(v.values(), []) 
                          for v in GEOGRAPHIC_VARS.values()], []),
        'Demographic': sum([sum(v.values(), []) for v in DEMOGRAPHIC_VARS.values()], []),
        'Activity': sum(ACTIVITY_VARS.values(), []),
        'Target': sum(TARGET_VARS['primary'].values(), []),
        'ID': ID_VARS['identifiers']
    }
    
    # 2. Vérifier les répétitions entre groupes
    all_vars = set(train_df.columns)
    classified_vars = set()
    duplicates = {}
    
    for group, vars_list in group_vars.items():
        # Vérifier les répétitions internes au groupe
        internal_dupes = [v for v in vars_list if vars_list.count(v) > 1]
        if internal_dupes:
            print(f"\nRépétitions internes dans {group}:")
            print(set(internal_dupes))
        
        # Vérifier les répétitions avec d'autres groupes
        for v in vars_list:
            if v in classified_vars:
                duplicates[v] = duplicates.get(v, []) + [group]
            classified_vars.add(v)
    
    # 3. Identifier les variables non classées
    unclassified = all_vars - classified_vars
    
    # 4. Afficher les résultats
    print("\n=== Résultats ===")
    print(f"Total variables dans DataFrame: {len(all_vars)}")
    print(f"Total variables classifiées: {len(classified_vars)}")
    print(f"Variables non classées: {len(unclassified)}")
    
    if duplicates:
        print("\nVariables présentes dans plusieurs groupes:")
        for var, groups in duplicates.items():
            print(f"- {var}: {groups}")
    
    if unclassified:
        print("\nVariables non classées:")
        print(sorted(unclassified))
    
    return {
        'group_stats': {group: len(vars_) for group, vars_ in group_vars.items()},
        'duplicates': duplicates,
        'unclassified': unclassified
    }

# Exécuter la vérification
classification_results = verify_variable_classification()

In [ ]:
# Cellule 1 : Fonction utilitaire
def flatten_vars(var_dict):
    """
    Aplatit un dictionnaire imbriqué de variables en une liste.
    """
    flat_list = []
    for k, v in var_dict.items():
        if isinstance(v, dict):
            for sub_v in v.values():
                if isinstance(sub_v, list):
                    flat_list.extend(sub_v)
                elif isinstance(sub_v, dict):
                    flat_list.extend(sum(sub_v.values(), []))
        elif isinstance(v, list):
            flat_list.extend(v)
    return flat_list

In [ ]:
# Cellule 2 : Analyse initiale
def analyze_missing_values(df):
    """
    Analyse détaillée des valeurs manquantes par groupe et par tranche.
    """
    groups = {
        'Weather': sum(WEATHER_VARS.values(), []),
        'Building': sum(BUILDING_VARS.values(), []),
        'Activity': flatten_vars(ACTIVITY_VARS),
        'Insurance': flatten_vars(INSURANCE_VARS),
        'Geographic': flatten_vars(GEOGRAPHIC_VARS),
        'Demographic': flatten_vars(DEMOGRAPHIC_VARS)
    }
    
    missing_summary = pd.DataFrame({
        'Missing Count': df.isnull().sum(),
        'Missing Percentage': df.isnull().sum() / len(df) * 100
    }).sort_values('Missing Percentage', ascending=False)
    
    print_missing_analysis(df, missing_summary, groups)
    
    return missing_summary, groups

def print_missing_analysis(df, missing_summary, groups):
    """Affiche l'analyse des valeurs manquantes"""
    ranges = {
        '0%': (0, 0),
        '< 10%': (0, 10),
        '10-30%': (10, 30),
        '30-50%': (30, 50),
        '50-70%': (50, 70),
        '70-90%': (70, 90),
        '> 90%': (90, 100)
    }
    
    total_cols = 0
    print("=== Analyse des valeurs manquantes par tranche ===\n")
    
    for range_name, (min_val, max_val) in ranges.items():
        range_cols = missing_summary[
            missing_summary['Missing Percentage'].between(min_val, max_val, inclusive='right' if range_name != '0%' else 'both')
        ]
        
        n_cols = len(range_cols)
        total_cols += n_cols
        
        if n_cols > 0:
            print(f"\n=== Tranche {range_name} ({n_cols} colonnes) ===")
            for group_name, cols in groups.items():
                group_cols = [col for col in range_cols.index if col in cols]
                if group_cols:
                    print(f"\n{group_name}: {len(group_cols)} colonnes")
                    print(f"Variables: {group_cols}")
    
    print(f"\nTotal des colonnes analysées: {total_cols}")
    print(f"Nombre total de colonnes: {len(df.columns)}")

In [ ]:
# Cellule 3 : Préparation des données
exclude_cols = ['FREQ', 'CM', 'CHARGE']
X_train = train_df.drop(exclude_cols, axis=1)
y_train = train_df[['FREQ', 'CM', 'CHARGE']]
X_test = test_df.copy()

# Analyse initiale
missing_summary, groups = analyze_missing_values(X_train)

In [ ]:
# Cellule 4 : Traitement Weather
def clean_weather_value(value):
    """
    Nettoie une valeur météo en extrayant le nombre après '<=' ou '>='
    Ex: '02. <= 65' -> 65
    Ex: '04. >= 7' -> 7
    """
    if pd.isna(value):
        return value
    try:
        # Enlever le préfixe (ex: "02. ")
        value = value.split('.')[1].strip()
        # Extraire le nombre après '<=' ou '>='
        if '<=' in value:
            return float(value.split('<=')[1].strip())
        elif '>=' in value:
            return float(value.split('>=')[1].strip())
        return value
    except:
        return value

def  clean_weather_vars(X_train, X_test, missing_summary, weather_cols):
    """
    Traite les variables météo avec conversion des valeurs catégorielles en numériques
    """
    missing_indicators = {}
    print("\nTraitement des variables météo:")
    
    for col in weather_cols:
        if col in X_train.columns:
            missing_pct = missing_summary.loc[col, 'Missing Percentage']
            print(f"\nColonne {col}:")
            print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
            print(f"- Type initial: {X_train[col].dtype}")
            
            # Afficher quelques valeurs avant conversion
            if X_train[col].dtype == 'object':
                print("- Exemples de valeurs avant conversion:")
                print(X_train[col].dropna().unique()[:5])
            
            # Créer indicateur de valeurs manquantes
            if missing_pct > 0:
                missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
            
            if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                # Convertir les valeurs catégorielles en numériques
                if X_train[col].dtype == 'object':
                    print("- Conversion en numérique")
                    X_train[col] = X_train[col].apply(clean_weather_value)
                    X_test[col] = X_test[col].apply(clean_weather_value)
                    print(f"- Type après conversion: {X_train[col].dtype}")
                
                # Imputation avec la médiane
                median_val = X_train[col].median()
                X_train[col].fillna(median_val, inplace=True)
                X_test[col].fillna(median_val, inplace=True)
                print(f"- Imputation avec la médiane: {median_val}")
                
                # Vérification des valeurs uniques après traitement
                unique_vals = sorted(X_train[col].unique())[:5]
                print(f"- Exemples de valeurs après traitement: {unique_vals}")
            else:
                print("- Colonne ignorée (>70% manquant)")
    
    return X_train, X_test, missing_indicators

# Exécution
X_train, X_test, weather_indicators = handle_weather_vars(
    X_train, X_test, missing_summary, groups['Weather']
)

In [ ]:
# Cellule 5 : Traitement Building
def handle_building_vars(X_train, X_test, missing_summary, building_cols):
    """
    Traite les variables bâtiment avec vérification détaillée des types
    """
    missing_indicators = {}
    print("\nTraitement des variables bâtiment:")
    
    for col in building_cols:
        if col in X_train.columns:
            missing_pct = missing_summary.loc[col, 'Missing Percentage']
            print(f"\nColonne {col}:")
            print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
            print(f"- Type initial: {X_train[col].dtype}")
            
            # Créer indicateur de valeurs manquantes
            if missing_pct > 0:
                missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
            
            if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                if X_train[col].dtype in ['int64', 'float64']:
                    # Pour les variables numériques
                    median_val = X_train[col].median()
                    X_train[col].fillna(median_val, inplace=True)
                    X_test[col].fillna(median_val, inplace=True)
                    print(f"- Variable numérique, imputation avec la médiane: {median_val}")
                    print(f"- Plage de valeurs: [{X_train[col].min()}, {X_train[col].max()}]")
                else:
                    # Pour les variables catégorielles
                    # Filtrer les valeurs non-nulles avant de les afficher
                    unique_vals = X_train[col].dropna().unique()
                    print("- Variable catégorielle, valeurs uniques:")
                    print(unique_vals[:5])
                    
                    # Si peu de valeurs uniques, on pourrait considérer une conversion en numérique
                    if len(unique_vals) < 10:
                        print(f"- Nombre de catégories: {len(unique_vals)}")
                        print("- Distribution des valeurs:")
                        print(X_train[col].value_counts().head())
                    
                    mode_val = X_train[col].mode()[0]
                    X_train[col].fillna('UNKNOWN', inplace=True)
                    X_test[col].fillna('UNKNOWN', inplace=True)
                    print(f"- Imputation avec 'UNKNOWN'")
            else:
                print("- Colonne ignorée (>70% manquant)")
            
            # Vérification après traitement
            missing_after = X_train[col].isnull().sum()
            if missing_after > 0:
                print(f"⚠️ ATTENTION: {missing_after} valeurs manquantes restantes!")
    
    return X_train, X_test, missing_indicators

# Exécution
X_train, X_test, building_indicators = handle_building_vars(
    X_train, X_test, missing_summary, groups['Building']
)

In [ ]:
# Cellule 6 : Traitement Activity
def handle_activity_vars(X_train, X_test, missing_summary, activity_cols):
    """
    Traite les variables d'activité avec vérification détaillée des types et valeurs
    """
    missing_indicators = {}
    print("\nTraitement des variables d'activité:")
    
    for col in activity_cols:
        if col in X_train.columns:
            missing_pct = missing_summary.loc[col, 'Missing Percentage']
            print(f"\nColonne {col}:")
            print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
            print(f"- Type initial: {X_train[col].dtype}")
            
            # Créer indicateur de valeurs manquantes
            if missing_pct > 0:
                missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
            
            if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                if X_train[col].dtype in ['int64', 'float64']:
                    # Pour les variables numériques
                    median_val = X_train[col].median()
                    X_train[col].fillna(median_val, inplace=True)
                    X_test[col].fillna(median_val, inplace=True)
                    print(f"- Variable numérique, imputation avec la médiane: {median_val}")
                    print(f"- Plage de valeurs: [{X_train[col].min()}, {X_train[col].max()}]")
                    print(f"- Distribution: \n{X_train[col].value_counts().head()}")
                else:
                    # Pour les variables catégorielles
                    unique_vals = X_train[col].unique()
                    print(f"- Variable catégorielle")
                    print(f"- Nombre de catégories uniques: {len(unique_vals)}")
                    print(f"- Top 5 catégories les plus fréquentes:")
                    print(X_train[col].value_counts().head())
                    
                    mode_val = X_train[col].mode()[0]
                    X_train[col].fillna('UNKNOWN', inplace=True)
                    X_test[col].fillna('UNKNOWN', inplace=True)
                    print(f"- Imputation avec 'UNKNOWN'")
            else:
                print("- Colonne ignorée (>70% manquant)")
            
            # Vérification après traitement
            missing_after = X_train[col].isnull().sum()
            if missing_after > 0:
                print(f"⚠️ ATTENTION: {missing_after} valeurs manquantes restantes!")
            
            # Vérification de la cohérence entre train et test
            train_unique = set(X_train[col].unique())
            test_unique = set(X_test[col].unique())
            diff_cats = test_unique - train_unique
            if len(diff_cats) > 0:
                print(f"⚠️ ATTENTION: Catégories présentes dans test mais pas dans train: {diff_cats}")
    
    return X_train, X_test, missing_indicators

# Exécution
X_train, X_test, activity_indicators = handle_activity_vars(
    X_train, X_test, missing_summary, groups['Activity']
)

In [ ]:
# Cellule 7 : Traitement Insurance
def handle_insurance_vars(X_train, X_test, missing_summary, insurance_cols):
    """
    Traite les variables d'assurance avec vérification détaillée des types et valeurs
    """
    missing_indicators = {}
    print("\nTraitement des variables d'assurance:")
    
    for col in insurance_cols:
        if col in X_train.columns:
            missing_pct = missing_summary.loc[col, 'Missing Percentage']
            print(f"\nColonne {col}:")
            print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
            print(f"- Type initial: {X_train[col].dtype}")
            
            # Créer indicateur de valeurs manquantes
            if missing_pct > 0:
                missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
            
            if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                if X_train[col].dtype in ['int64', 'float64']:
                    # Pour les variables numériques
                    median_val = X_train[col].median()
                    mean_val = X_train[col].mean()
                    std_val = X_train[col].std()
                    
                    print(f"- Variable numérique:")
                    print(f"  * Médiane: {median_val}")
                    print(f"  * Moyenne: {mean_val:.2f}")
                    print(f"  * Écart-type: {std_val:.2f}")
                    print(f"  * Plage: [{X_train[col].min()}, {X_train[col].max()}]")
                    
                    # Détection des valeurs aberrantes
                    outliers = X_train[col][(X_train[col] > mean_val + 3*std_val) | 
                                         (X_train[col] < mean_val - 3*std_val)].count()
                    if outliers > 0:
                        print(f"⚠️ {outliers} valeurs aberrantes détectées (>3σ)")
                    
                    X_train[col].fillna(median_val, inplace=True)
                    X_test[col].fillna(median_val, inplace=True)
                    
                else:
                    # Pour les variables catégorielles
                    unique_vals = X_train[col].unique()
                    print(f"- Variable catégorielle:")
                    print(f"  * Nombre de catégories: {len(unique_vals)}")
                    print("  * Distribution des valeurs:")
                    print(X_train[col].value_counts().head())
                    
                    # Si la variable a peu de catégories, on pourrait envisager un encodage
                    if len(unique_vals) < 10:
                        print("  * Candidat potentiel pour encodage catégoriel")
                    
                    X_train[col].fillna('UNKNOWN', inplace=True)
                    X_test[col].fillna('UNKNOWN', inplace=True)
            else:
                print("- Colonne ignorée (>70% manquant)")
            
            # Vérifications post-traitement
            missing_after = X_train[col].isnull().sum()
            if missing_after > 0:
                print(f"⚠️ ATTENTION: {missing_after} valeurs manquantes restantes!")
            
            # Vérification de la cohérence train/test
            if X_train[col].dtype == 'object':
                train_cats = set(X_train[col].unique())
                test_cats = set(X_test[col].unique())
                new_cats = test_cats - train_cats
                if new_cats:
                    print(f"⚠️ Nouvelles catégories dans test: {new_cats}")
    
    return X_train, X_test, missing_indicators

# Exécution
X_train, X_test, insurance_indicators = handle_insurance_vars(
    X_train, X_test, missing_summary, groups['Insurance']
)

In [ ]:
def handle_geographic_vars(X_train, X_test, missing_summary, geographic_cols):
    """
    Traite les variables géographiques avec vérification détaillée des types et valeurs
    """
    missing_indicators = {}
    print("\nTraitement des variables géographiques:")
    
    # Regrouper les colonnes par type
    distance_cols = [col for col in geographic_cols if 'DISTANCE' in col]
    proportion_cols = [col for col in geographic_cols if 'PROPORTION' in col]
    altitude_cols = [col for col in geographic_cols if 'ALTITUDE' in col]
    other_cols = [col for col in geographic_cols if col not in distance_cols + proportion_cols + altitude_cols]
    
    for col_type, cols in [
        ("Distance", distance_cols),
        ("Proportion", proportion_cols),
        ("Altitude", altitude_cols),
        ("Autres", other_cols)
    ]:
        print(f"\n=== Variables de {col_type} ===")
        
        for col in cols:
            if col in X_train.columns:
                missing_pct = missing_summary.loc[col, 'Missing Percentage']
                print(f"\nColonne {col}:")
                print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
                print(f"- Type initial: {X_train[col].dtype}")
                
                # Créer indicateur de valeurs manquantes
                if missing_pct > 0:
                    missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
                
                if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                    if X_train[col].dtype in ['int64', 'float64']:
                        # Pour les variables numériques
                        stats = X_train[col].describe()
                        print("- Statistiques descriptives:")
                        print(f"  * Moyenne: {stats['mean']:.2f}")
                        print(f"  * Écart-type: {stats['std']:.2f}")
                        print(f"  * Min: {stats['min']:.2f}")
                        print(f"  * 25%: {stats['25%']:.2f}")
                        print(f"  * Médiane: {stats['50%']:.2f}")
                        print(f"  * 75%: {stats['75%']:.2f}")
                        print(f"  * Max: {stats['max']:.2f}")
                        
                        # Vérification des valeurs négatives pour les distances
                        if 'DISTANCE' in col and (X_train[col] < 0).any():
                            print("⚠️ ATTENTION: Valeurs négatives détectées pour une distance!")
                        
                        # Vérification des proportions
                        if 'PROPORTION' in col:
                            if (X_train[col] < 0).any() or (X_train[col] > 1).any():
                                print("⚠️ ATTENTION: Proportions hors de l'intervalle [0,1]!")
                        
                        median_val = stats['50%']
                        X_train[col].fillna(median_val, inplace=True)
                        X_test[col].fillna(median_val, inplace=True)
                        print(f"- Imputation avec la médiane: {median_val}")
                        
                    else:
                        # Pour les variables catégorielles
                        # Extraire le nombre après '<=' ou '>='
                        def extract_number(x):
                            if pd.isna(x):
                                return x
                            try:
                                if '<=' in str(x):
                                    return float(str(x).split('<=')[1].strip())
                                elif '>=' in str(x):
                                    return float(str(x).split('>=')[1].strip())
                                return float(x)
                            except:
                                return x
                        
                        print("- Conversion en numérique...")
                        X_train[col] = X_train[col].apply(extract_number)
                        X_test[col] = X_test[col].apply(extract_number)
                        
                        # Imputation avec la médiane après conversion
                        median_val = X_train[col].median()
                        X_train[col].fillna(median_val, inplace=True)
                        X_test[col].fillna(median_val, inplace=True)
                        print(f"- Imputation avec la médiane: {median_val}")
                        
                else:
                    print("- Colonne ignorée (>70% manquant)")
                
                # Vérifications post-traitement
                missing_after = X_train[col].isnull().sum()
                if missing_after > 0:
                    print(f"⚠️ ATTENTION: {missing_after} valeurs manquantes restantes!")
    
    return X_train, X_test, missing_indicators

In [ ]:
# Cellule 9 : Traitement Demographic
def handle_demographic_vars(X_train, X_test, missing_summary, demographic_cols):
    """
    Traite les variables démographiques avec vérification détaillée des types et valeurs
    """
    missing_indicators = {}
    print("\nTraitement des variables démographiques:")
    
    # Regrouper les colonnes par type
    household_cols = [col for col in demographic_cols if col.startswith('MEN')]
    individual_cols = [col for col in demographic_cols if col.startswith('IND')]
    housing_cols = [col for col in demographic_cols if col.startswith('LOG')]
    
    for col_type, cols in [
        ("Ménages", household_cols),
        ("Individus", individual_cols),
        ("Logements", housing_cols)
    ]:
        print(f"\n=== Variables {col_type} ===")
        
        for col in cols:
            if col in X_train.columns:
                missing_pct = missing_summary.loc[col, 'Missing Percentage']
                print(f"\nColonne {col}:")
                print(f"- Pourcentage de valeurs manquantes: {missing_pct:.2f}%")
                print(f"- Type initial: {X_train[col].dtype}")
                
                # Créer indicateur de valeurs manquantes
                if missing_pct > 0:
                    missing_indicators[f'{col}_is_missing'] = X_train[col].isnull().astype(int)
                
                if missing_pct <= 70:  # On traite uniquement si moins de 70% manquant
                    if X_train[col].dtype in ['int64', 'float64']:
                        # Pour les variables numériques
                        stats = X_train[col].describe()
                        print("- Statistiques descriptives:")
                        print(f"  * Moyenne: {stats['mean']:.2f}")
                        print(f"  * Écart-type: {stats['std']:.2f}")
                        print(f"  * Min: {stats['min']:.2f}")
                        print(f"  * 25%: {stats['25%']:.2f}")
                        print(f"  * Médiane: {stats['50%']:.2f}")
                        print(f"  * 75%: {stats['75%']:.2f}")
                        print(f"  * Max: {stats['max']:.2f}")
                        
                        # Vérifications spécifiques selon le type de variable
                        if col.startswith('MEN') or col.startswith('IND'):
                            if (X_train[col] < 0).any():
                                print("⚠️ ATTENTION: Valeurs négatives détectées pour un comptage!")
                        
                        # Vérification de la cohérence des proportions
                        if 'PROP' in col or 'PAUV' in col:
                            if (X_train[col] < 0).any() or (X_train[col] > 1).any():
                                print("⚠️ ATTENTION: Proportions hors de l'intervalle [0,1]!")
                        
                        median_val = stats['50%']
                        X_train[col].fillna(median_val, inplace=True)
                        X_test[col].fillna(median_val, inplace=True)
                        print(f"- Imputation avec la médiane: {median_val}")
                        
                    else:
                        # Pour les variables catégorielles
                        value_counts = X_train[col].value_counts()
                        print("- Distribution des valeurs:")
                        print(value_counts.head())
                        print(f"- Nombre total de catégories: {len(value_counts)}")
                        
                        X_train[col].fillna('UNKNOWN', inplace=True)
                        X_test[col].fillna('UNKNOWN', inplace=True)
                        print("- Imputation avec 'UNKNOWN'")
                else:
                    print("- Colonne ignorée (>70% manquant)")
                
                # Vérifications post-traitement
                missing_after = X_train[col].isnull().sum()
                if missing_after > 0:
                    print(f"⚠️ ATTENTION: {missing_after} valeurs manquantes restantes!")
                
                # Vérification de la cohérence des données
                if col.startswith('MEN'):
                    total_men = X_train['MEN'] if 'MEN' in X_train.columns else None
                    if total_men is not None and col != 'MEN':
                        if (X_train[col] > total_men).any():
                            print("⚠️ ATTENTION: Sous-catégorie de ménages supérieure au total!")
                
                if col.startswith('IND'):
                    total_ind = X_train['IND'] if 'IND' in X_train.columns else None
                    if total_ind is not None and col != 'IND':
                        if (X_train[col] > total_ind).any():
                            print("⚠️ ATTENTION: Sous-catégorie d'individus supérieure au total!")
    
    return X_train, X_test, missing_indicators

# Exécution
X_train, X_test, demographic_indicators = handle_demographic_vars(
    X_train, X_test, missing_summary, groups['Demographic']
)

In [ ]:
# Cellule 10 : Finalisation
def finalize_processing(X_train, X_test, all_indicators, missing_summary):
    """
    Finalise le traitement des données avec vérifications détaillées
    """
    print("\n=== Début de la finalisation ===")
    
    # État initial
    print("\nÉtat initial:")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    print(f"Nombre d'indicateurs: {len(all_indicators)}")
    
    # Vérification des index
    print("\nVérification des index:")
    print(f"Index train unique: {X_train.index.is_unique}")
    print(f"Index test unique: {X_test.index.is_unique}")
    
    # Combiner les indicateurs
    print("\nCréation des DataFrames d'indicateurs...")
    indicators_df_train = pd.DataFrame(all_indicators, index=X_train.index)
    indicators_df_test = pd.DataFrame(all_indicators, index=X_test.index)
    
    print(f"Shape indicateurs train: {indicators_df_train.shape}")
    print(f"Shape indicateurs test: {indicators_df_test.shape}")
    
    # Concaténation
    print("\nConcaténation avec les données principales...")
    X_train = pd.concat([X_train, indicators_df_train], axis=1)
    X_test = pd.concat([X_test, indicators_df_test], axis=1)
    
    # Suppression des colonnes avec beaucoup de valeurs manquantes
    high_missing = missing_summary[missing_summary['Missing Percentage'] > 70].index
    print(f"\nSuppression de {len(high_missing)} colonnes avec >70% de valeurs manquantes:")
    print(high_missing.tolist())
    
    X_train.drop(high_missing, axis=1, inplace=True)
    X_test.drop(high_missing, axis=1, inplace=True)
    
    # Vérifications finales
    print("\n=== État final des données ===")
    print(f"Shape après traitement:")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    
    # Vérification des valeurs manquantes
    final_missing_train = X_train.isnull().sum()
    final_missing_test = X_test.isnull().sum()
    
    if final_missing_train.sum() > 0:
        print("\nColonnes avec valeurs manquantes restantes dans train:")
        print(final_missing_train[final_missing_train > 0])
    else:
        print("\nAucune valeur manquante restante dans train")
        
    if final_missing_test.sum() > 0:
        print("\nColonnes avec valeurs manquantes restantes dans test:")
        print(final_missing_test[final_missing_test > 0])
    else:
        print("\nAucune valeur manquante restante dans test")
    
    # Vérification des types de données
    print("\nTypes de données dans le jeu final:")
    print(X_train.dtypes.value_counts())
    
    # Vérification de la cohérence train/test
    train_cols = set(X_train.columns)
    test_cols = set(X_test.columns)
    
    if train_cols != test_cols:
        print("\n⚠️ ATTENTION: Différences dans les colonnes train/test!")
        print("Colonnes uniquement dans train:", train_cols - test_cols)
        print("Colonnes uniquement dans test:", test_cols - train_cols)
    else:
        print("\nColonnes cohérentes entre train et test")
    
    return X_train, X_test

# Initialiser le dictionnaire des indicateurs
all_indicators = {}

# Liste des groupes d'indicateurs à vérifier
indicator_groups = {
    'weather': ('weather_indicators', weather_indicators if 'weather_indicators' in globals() else {}),
    'building': ('building_indicators', building_indicators if 'building_indicators' in globals() else {}),
    'activity': ('activity_indicators', activity_indicators if 'activity_indicators' in globals() else {}),
    'insurance': ('insurance_indicators', insurance_indicators if 'insurance_indicators' in globals() else {}),
    'geographic': ('geographic_indicators', geographic_indicators if 'geographic_indicators' in globals() else {}),
    'demographic': ('demographic_indicators', demographic_indicators if 'demographic_indicators' in globals() else {})
}

# Ajouter chaque groupe d'indicateurs disponible
for group_name, (var_name, indicators) in indicator_groups.items():
    if indicators:
        print(f"Ajout de {len(indicators)} indicateurs du groupe {group_name}")
        all_indicators.update(indicators)
    else:
        print(f"⚠️ Attention: Les indicateurs {group_name} sont manquants ou vides")

# Traitement final
X_train_processed, X_test_processed = finalize_processing(
    X_train, X_test, all_indicators, missing_summary
)